In [1]:
# Import packages
import pandas as pd
import os

In [2]:
# Import useful files
df_agency = pd.read_csv("agency.txt")
df_calendar = pd.read_csv("calendar.txt")
df_shapes = pd.read_csv("shapes.txt")
df_transfers = pd.read_csv("transfers.txt")
df_calendarDates = pd.read_csv("calendar_dates.txt")
df_routes = pd.read_csv("routes.txt")
df_stops = pd.read_csv("stops.txt")
df_trips = pd.read_csv("trips.txt")
df_stopTimes = pd.read_csv("stop_times.txt")

In [3]:
# Shortlist services only on weekends
df_calendar['flag']= df_calendar.apply(lambda x: (x['saturday']>0)|(x['sunday']>0), axis=1)
df_servicesWeekend=df_calendar.loc[df_calendar['flag']==True,['service_id','flag']]
df_servicesWeekend.head()

,service_id,flag
0,AFA18GEN-1037-Sunday-00,True
1,AFA18GEN-2048-Sunday-00,True
2,AFA18GEN-3041-Sunday-00,True
3,AFA18GEN-4049-Sunday-00,True
4,AFA18GEN-5048-Sunday-00,True


In [15]:
# Shortlist trips on weekends
df_tripsWeekend=df_trips.merge(df_servicesWeekend, on='service_id', how="inner")
df_tripsWeekend=df_tripsWeekend.loc[:,['route_id', 'trip_id', 'trip_headsign','direction_id','service_id']]
df_tripsWeekend['New_trip_id']=df_tripsWeekend.apply(lambda x: x['trip_id'].split('.')[len(x['trip_id'].split('.'))-1], axis=1)
df_tripsWeekend.head()

,route_id,trip_id,trip_headsign,direction_id,service_id,New_trip_id
0,1,AFA18GEN-1037-Sunday-00_000600_1..S03R,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
1,1,AFA18GEN-1037-Sunday-00_002600_1..S03R,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
2,1,AFA18GEN-1037-Sunday-00_004600_1..S03R,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
3,1,AFA18GEN-1037-Sunday-00_006600_1..S03R,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
4,1,AFA18GEN-1037-Sunday-00_007200_1..N03R,Van Cortlandt Park - 242 St,0.0,AFA18GEN-1037-Sunday-00,N03R


In [50]:
# Shortlist stops times applicable on weekends
df_stopTimesWeekend=df_stopTimes.merge(df_tripsWeekend, on='trip_id', how="inner")
df_stopTimesWeekend=df_stopTimesWeekend[['stop_id','stop_sequence','route_id','trip_headsign','direction_id','service_id','New_trip_id']]
print(len(df_stopTimesWeekend))
df_stopTimesWeekend=df_stopTimesWeekend.drop_duplicates()
print(len(df_stopTimesWeekend))
df_stopTimesWeekend['stop_id']=df_stopTimesWeekend.apply(lambda x: x['stop_id'][0:3], axis=1)
df_stopTimesWeekend.head()

450465
5057


,stop_id,stop_sequence,route_id,trip_headsign,direction_id,service_id,New_trip_id
0,101,1,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
1,103,2,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
2,104,3,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
3,106,4,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R
4,107,5,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R


In [51]:
# Define function for determining pre and post stops
def stopSequence(row):
    pre = row['stop_sequence']-1
    try:
        pre_stop = list(df_stopTimesWeekend.loc[(df_stopTimesWeekend['stop_sequence']==pre) & (df_stopTimesWeekend['service_id']==row['service_id']) & (df_stopTimesWeekend['New_trip_id']==row['New_trip_id']) ,'stop_id'])[0]
    except:
        pre_stop = ''
    
    post = row['stop_sequence']+1
    try:
        post_stop = list(df_stopTimesWeekend.loc[(df_stopTimesWeekend['stop_sequence']==post) & (df_stopTimesWeekend['service_id']==row['service_id']) & (df_stopTimesWeekend['New_trip_id']==row['New_trip_id']),'stop_id'])[0]
    except:
        post_stop = ''
    return pre_stop, post_stop


In [52]:
# Add pre and post stops to each stop
df_stopTimesWeekend['Pre_Stop'],df_stopTimesWeekend['Post_Stop']=zip(*df_stopTimesWeekend.apply( lambda row: stopSequence(row), axis=1))

In [54]:
# Subset final dataframe and remove duplicates
df_stopTimesWeekendFinal=df_stopTimesWeekend[['route_id','trip_headsign','direction_id', 'Pre_Stop','stop_id','Post_Stop','New_trip_id']]
print(len(df_stopTimesWeekendFinal))
df_stopTimesWeekendFinal=df_stopTimesWeekendFinal.drop_duplicates()
print(len(df_stopTimesWeekendFinal))

5057
2598


In [55]:
df_stopsSubset = df_stops[['stop_id','stop_name']]

In [56]:
def name(row, key):
    try:
        name = list(df_stopsSubset.loc[df_stopsSubset['stop_id']==row[key],'stop_name'])[0]
    except:
        name=''
    return name

In [57]:
df_stopTimesWeekendFinal['Pre_Stop_Name'] = df_stopTimesWeekendFinal.apply(lambda row: name(row, 'Pre_Stop'), axis=1)
df_stopTimesWeekendFinal['Stop_Name'] = df_stopTimesWeekendFinal.apply(lambda row: name(row, 'stop_id'), axis=1)
df_stopTimesWeekendFinal['Post_Stop_Name'] = df_stopTimesWeekendFinal.apply(lambda row: name(row, 'Post_Stop'), axis=1)

In [58]:
df_stopTimesWeekendFinal.to_csv("PrePostStops.csv")

#### EXTRACTING ROUTES SEQUENCE

In [59]:
df_stopTimesWeekendRoutes = df_stopTimesWeekend.drop_duplicates(['stop_id', 'stop_sequence', 'route_id', 'trip_headsign','New_trip_id'])
print(len(df_stopTimesWeekend))
print(len(df_stopTimesWeekendRoutes))
df_stopTimesWeekendRoutes.head()

5057
2598


,stop_id,stop_sequence,route_id,trip_headsign,direction_id,service_id,New_trip_id,Pre_Stop,Post_Stop
0,101,1,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R,,103
1,103,2,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R,101,104
2,104,3,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R,103,106
3,106,4,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R,104,107
4,107,5,1,South Ferry,1.0,AFA18GEN-1037-Sunday-00,S03R,106,108


In [60]:
df_stopTimesWeekendRoutesFinal=df_stopTimesWeekendRoutes[['route_id','New_trip_id']].drop_duplicates()
print(len(df_stopTimesWeekendRoutesFinal))
df_stopTimesWeekendRoutesFinal.head()

93


,route_id,New_trip_id
0,1,S03R
38,1,N03R
22192,2,S08R
23107,2,S01R
27627,2,N08R


In [72]:
def route(row):
    df_temp = df_stopTimesWeekendRoutes.loc[(df_stopTimesWeekendRoutes['route_id']==row['route_id']) & (df_stopTimesWeekendRoutes['New_trip_id']==row['New_trip_id']),:]
    lis=[]
    for i in range(1, df_temp['stop_sequence'].max()+1):
        lis.append(list(df_temp.loc[df_temp['stop_sequence']==i, 'stop_id'])[0])
    return lis

In [73]:
df_stopTimesWeekendRoutesFinal['Route']=df_stopTimesWeekendRoutesFinal.apply(lambda row: route(row), axis=1)

In [75]:
df_stopTimesWeekendRoutesFinal.to_csv("Routes.csv")

#### 2018-11-23: Part 2: ONLY F N R Lines

In [127]:
# Import packages
import pandas as pd
import os

In [128]:
df_main = pd.read_csv("20181123_PrePostStops.csv")

In [129]:
print(len(df_main))
df_main = df_main.drop_duplicates(subset=['route_id','direction_id','Pre_Stop','stop_id','Post_Stop'])
print(len(df_main))
df_main = df_main.fillna('-')
df_main.head()

411
282


,route_id,trip_headsign,direction_id,Pre_Stop,stop_id,Post_Stop,New_trip_id,Pre_Stop_Name,Stop_Name,Post_Stop_Name,Flag,Shuttle
0,F,Jamaica - 179 St,Q,-,D43,D42,N69R,-,Coney Island - Stillwell Av,W 8 St - NY Aquarium,x,0
1,F,Jamaica - 179 St,Q,D43,D42,F39,N69R,Coney Island - Stillwell Av,W 8 St - NY Aquarium,Neptune Av,x,0
2,F,Jamaica - 179 St,Q,D42,F39,F38,N69R,W 8 St - NY Aquarium,Neptune Av,Avenue X,x,0
3,F,Jamaica - 179 St,Q,F39,F38,F36,N69R,Neptune Av,Avenue X,Avenue U,x,0
4,F,Jamaica - 179 St,Q,F38,F36,F35,N69R,Avenue X,Avenue U,Kings Hwy,x,0


In [145]:
df_main['String1']=df_main.apply(lambda x: list(x['route_id'])[0]+":"+list(x['direction_id'])[0]+":"+x['Pre_Stop']+":"+x['stop_id']+":x:s ;", axis=1)
df_main['String2']=df_main.apply(lambda x: list(x['route_id'])[0]+":"+list(x['direction_id'])[0]+":"+x['stop_id']+":"+x['Post_Stop']+":x:s ;", axis=1)
df_main.head()

,route_id,trip_headsign,direction_id,Pre_Stop,stop_id,Post_Stop,New_trip_id,Pre_Stop_Name,Stop_Name,Post_Stop_Name,Flag,Shuttle,String1,String2
0,F,Jamaica - 179 St,Q,-,D43,D42,N69R,-,Coney Island - Stillwell Av,W 8 St - NY Aquarium,x,0,F:Q:-:D43:x:s ;,F:Q:D43:D42:x:s ;
1,F,Jamaica - 179 St,Q,D43,D42,F39,N69R,Coney Island - Stillwell Av,W 8 St - NY Aquarium,Neptune Av,x,0,F:Q:D43:D42:x:s ;,F:Q:D42:F39:x:s ;
2,F,Jamaica - 179 St,Q,D42,F39,F38,N69R,W 8 St - NY Aquarium,Neptune Av,Avenue X,x,0,F:Q:D42:F39:x:s ;,F:Q:F39:F38:x:s ;
3,F,Jamaica - 179 St,Q,F39,F38,F36,N69R,Neptune Av,Avenue X,Avenue U,x,0,F:Q:F39:F38:x:s ;,F:Q:F38:F36:x:s ;
4,F,Jamaica - 179 St,Q,F38,F36,F35,N69R,Avenue X,Avenue U,Kings Hwy,x,0,F:Q:F38:F36:x:s ;,F:Q:F36:F35:x:s ;


In [146]:
df_strings1 = df_main[['stop_id', 'String1']]
df_strings1.rename(columns={'String1':'String'},inplace=True)
df_strings2 = df_main[['stop_id', 'String2']]
df_strings2.rename(columns={'String2':'String'},inplace=True)
df_strings = df_strings1.append(df_strings2)
df_strings = df_strings.reset_index(drop=True)


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/ADS/lib/python3.5/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [147]:
df_strings

,stop_id,String
0,D43,F:Q:-:D43:x:s ;
1,D42,F:Q:D43:D42:x:s ;
2,F39,F:Q:D42:F39:x:s ;
3,F38,F:Q:F39:F38:x:s ;
4,F36,F:Q:F38:F36:x:s ;
5,F35,F:Q:F36:F35:x:s ;
6,F34,F:Q:F35:F34:x:s ;
7,F33,F:Q:F34:F33:x:s ;
8,F32,F:Q:F33:F32:x:s ;
9,F31,F:Q:F32:F31:x:s ;


In [148]:
df_final = pd.DataFrame()
for i in df_strings['stop_id'].unique():
    df_subset = df_strings.loc[df_strings['stop_id']==i,:]
    df_temp = pd.DataFrame()
    df_temp.loc[0,'stop_id'] = i 
    a=''
    for j in range(len(df_subset)):
        a = a + list(df_subset['String'])[j]
    df_temp.loc[0,'Dict'] = a
    df_final = df_final.append(df_temp)

In [149]:
df_final.to_csv('20181123_Dict.csv')